In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import datetime

In [ ]:
def weighted_column_addition(s_df):
    no_of_sample = len(s_df)
    s_df['Weight'] = np.nan
    
    if no_of_sample == 3:
        s_df['Weight'] = np.where(s_df['Relative Depth'] == 0.8, 0.4, s_df['Weight'])
        s_df['Weight'] = np.where(s_df['Relative Depth'] == 0.6, 0.3, s_df['Weight'])
        s_df['Weight'] = np.where(s_df['Relative Depth'] == 0.2, 0.3, s_df['Weight'])
    
    elif no_of_sample == 2:
        depths = sorted(s_df['Relative Depth'].tolist(), reverse=True)
    
        if depths == [0.8, 0.6]:
            s_df['Weight'] = np.where(s_df['Relative Depth'] == 0.8, 0.6, s_df['Weight'])
            s_df['Weight'] = np.where(s_df['Relative Depth'] == 0.6, 0.4, s_df['Weight'])
            
        elif depths == [0.8, 0.2]:
            s_df['Weight'] = np.where(s_df['Relative Depth'] == 0.8, 0.6, s_df['Weight'])
            s_df['Weight'] = np.where(s_df['Relative Depth'] == 0.2, 0.4, s_df['Weight'])
            
        elif depths == [0.6, 0.2]:
            s_df['Weight'] = np.where(s_df['Relative Depth'] == 0.6, 0.6, s_df['Weight'])
            s_df['Weight'] = np.where(s_df['Relative Depth'] == 0.2, 0.4, s_df['Weight'])
    
    elif no_of_sample == 1:
        s_df['Weight'] = 1
    
    return s_df


summary_df = pd.DataFrame()

input_file_folder_directory = r'..\..\Suspended_Sediment_Concentration'
ouput_file_directory = r'..\..\Suspended_Sediment_Concentration\SSC'


for file_path in glob.glob(os.path.join(input_file_folder_directory,"*.xlsx")):

    file_name = file_path.split(os.sep)[-1]
    file_name_list = file_name.split('_')
    river_name = f'{file_name_list[0]} {file_name_list[1]}'
    
    try:
        df = pd.read_excel(file_path, sheet_name = 'Processed')
        
        weighted_df = pd.DataFrame()
        
        for time in df['Date/Time'].unique():
            tc_df = df[df['Date/Time'] == time].copy()
        
            w_df = weighted_column_addition(tc_df)
            weighted_df = pd.concat([weighted_df, w_df])
        
        weighted_df.reset_index(drop=True, inplace=True)
        weighted_df['Weighted SSC (mg/l)'] = (weighted_df['SSC (mg/l)']) * (weighted_df['Weight'])
        selected_sc_df = weighted_df[['River', 'Date', 'Time', 'Easting (m)', 'Northing (m)', 'Weighted SSC (mg/l)']]
        
        avg_sc_df = selected_sc_df.groupby(by=['River', 'Date', 'Time', 'Easting (m)', 'Northing (m)'])['Weighted SSC (mg/l)'].sum().reset_index()
        daily_max_df = avg_sc_df.loc[avg_sc_df.groupby('Date')['Weighted SSC (mg/l)'].idxmax()]
        
        summary_df = pd.concat([summary_df, daily_max_df])

        output_file = os.path.join(ouput_file_directory, f"{file_name[:-5]}_Weighted.csv")
        weighted_df.to_csv(output_file, float_format = '%.4f', index=False)

    except:
        print(file_name)

In [33]:
summary_df.reset_index(drop=True, inplace=True)
summary_df

,River,Date,Time,Easting (m),Northing (m),Weighted SSC (mg/l)
0,Bhadra River,2025-04-13,09:00:00,748850.0,2506487.0,1907.615478
1,Bhadra River,2025-04-20,09:00:00,748850.0,2506487.0,296.635316
2,Chunkuri River,2025-04-15,08:00:00,759114.0,2501204.0,1814.607240
3,Chunkuri River,2025-04-22,11:00:00,759070.0,2501186.0,405.782839
4,Gengrail River,2025-04-28,10:00:00,746672.0,2507284.0,3166.848983
5,Gengrail River,2025-05-05,08:00:00,746681.0,2507280.0,750.455089
6,Hari River (DS),2025-04-29,15:00:00,741314.0,2526100.0,10357.703855
7,Hari River (DS),2025-05-04,13:00:00,741306.0,2526026.0,2523.075631
8,Hari River (US),2025-04-29,12:00:00,741299.0,2536928.0,64.841845
9,Hari River (US),2025-05-04,12:00:00,741307.0,2536929.0,81.362721


In [ ]:
summary_df.to_csv(r'..\..\Suspended_Sediment_Concentration\SSC\Dry_Season_Maximum_SSC_Summary_2025_V2.csv',
                  float_format = '%.4f', index=False)